In [17]:
import sys

sys.path.append("/home/daniil/repos/pytorch-segmentation-detection/")
sys.path.insert(0, '/home/daniil/repos/pytorch-segmentation-detection/vision/')

from pytorch_segmentation_detection.datasets.simple_dataset import SimpleDataset

test = SimpleDataset()



In [16]:
test_int = 5

In [18]:
int(str(test_int))


5

In [20]:
int(None)

TypeError: int() argument must be a string or a number, not 'NoneType'

In [24]:
import json


In [1]:
import numpy as np

In [9]:
dummy = np.zeros((100, 100, 3))

In [15]:
dummy.shape[:2]

(100, 100)

In [12]:
dummy[4:40, 4:40].shape

(36, 36, 3)

In [26]:
json.loads('')

ValueError: No JSON object could be decoded

In [23]:
not None

True

In [19]:
test

In [16]:
import os

os.remove(test.annotations_filenames[1])

In [5]:
test[11]

IOError: cannot identify image file '/home/daniil/.pytorch-segmentation-detection/datasets/simple_dataset/train/annotations/2018-07-28-15-56-11-912380.png'

In [6]:
from time import sleep

from PIL import Image

In [8]:
Image.open('/home/daniil/.pytorch-segmentation-detection/datasets/simple_dataset/train/annotations/2018-07-28-15-56-11-912380.png')

IOError: cannot identify image file '/home/daniil/.pytorch-segmentation-detection/datasets/simple_dataset/train/annotations/2018-07-28-15-56-11-912380.png'

In [2]:
import numpy as np

np.zeros((10, 10))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [13]:
from PIL import Image
import numpy as np

anno_pil = Image.open('/home/daniil/.pytorch-segmentation-detection/datasets/endovis_2017/instrument_dataset_1/ground_truth/Right_Prograsp_Forceps_labels/frame005.png')

In [18]:
anno_np = np.asarray(anno_pil).copy()


anno_np[anno_np > 0] = 1

anno_updated_pil = Image.fromarray(anno_np)

In [19]:
anno_updated_pil.save('/home/daniil/frame005_anno.png')

In [2]:
img, anno = test[0]

In [5]:
import numpy as np

anno_np = np.asarray( anno ).copy()
anno_np[:] = 1

In [6]:
np.unique( anno_np )

array([1], dtype=uint8)

In [10]:
from PIL import Image

test[2] =  Image.fromarray(anno_np)

In [ ]:
Image.fr

In [2]:
test.annotations_filenames

['/home/daniil/.pytorch-segmentation-detection/datasets/simple_dataset/train/annotations/2018-07-25-15-58-51-999436.png',
 '/home/daniil/.pytorch-segmentation-detection/datasets/simple_dataset/train/annotations/2018-07-25-15-59-36-545692.png',
 '/home/daniil/.pytorch-segmentation-detection/datasets/simple_dataset/train/annotations/2018-07-25-15-59-53-951964.png']

In [8]:
del test[2]

In [9]:
test.images_filenames

['/home/daniil/.pytorch-segmentation-detection/datasets/simple_dataset/train/images/2018-07-25-15-58-51-999436.png',
 '/home/daniil/.pytorch-segmentation-detection/datasets/simple_dataset/train/images/2018-07-25-15-59-36-545692.png',
 '/home/daniil/.pytorch-segmentation-detection/datasets/simple_dataset/train/images/2018-07-25-15-59-53-951964.png']

In [4]:
pair = test[2]

In [5]:
test.add_new_sample(pair)

In [5]:
test[2] = pair[1]

In [1]:
from dash_deep.models import EndovisBinary
from dash_deep.logging import Experiment
from random import random


experiment_model = EndovisBinary()
experiment_instance = Experiment(experiment_model)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [4]:
from time import sleep

epochs = range(10)
iterations = range(10)


for epoch in epochs:
    
    for iteration in iterations:

        experiment_instance.add_next_iteration_results(training_loss=random())
        
        sleep(5)
    
    experiment_instance.add_next_iteration_results(training_accuracy=random(),
                                                   validation_accuracy=random())
experiment_instance.finish()